In [ ]:
# add 'gdalwheel' dataset to input before running this command 
!python3 -m pip install ../input/gdalwheel/GDAL-3.1.4-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl

In [15]:
import numpy as np
from numpy import nan_to_num, subtract, add, divide, multiply
from osgeo import gdal, gdalconst
import gdal
from gdal import GetDriverByName, Open
from glob import glob
import os

In [10]:
def ndvi(in_nir_band, in_colour_band, in_rows, in_cols, in_geotransform, out_tiff, data_type=gdal.GDT_Float32):

    """
    Performs an NDVI calculation given two input bands, as well as other information that can be retrieved from the
    original image.
    @param in_nir_band A GDAL band object representing the near-infrared image data.
    @type in_nir_band GDALRasterBand
    @param in_colour_band A GDAL band object representing the colour image data.
    @type: in_colour_band GDALRasterBand
    @param in_rows The number of rows in both input bands.
    @type: in_rows int
    @param in_cols The number of columns in both input bands.
    @type: in_cols int
    @param in_geotransform The geographic transformation to be applied to the output image.
    @type in_geotransform Tuple (as returned by GetGeoTransform())
    @param out_tiff Path to the desired output .tif file.
    @type: out_tiff String (should end in ".tif")
    @param data_type Data type of output image.  Valid values are gdal.UInt16 and gdal.Float32.  Default is
                      gdal.Float32
    @type data_type GDALDataType
    @return None
    """

    # Read the input bands as numpy arrays.
    np_nir = in_nir_band.ReadAsArray(0, 0, in_cols, in_rows)
    np_colour = in_colour_band.ReadAsArray(0, 0, in_cols, in_rows)

    # Convert the np arrays to 32-bit floating point to make sure division will occur properly.
    np_nir_as32 = np_nir.astype(np.float32)
    np_colour_as32 = np_colour.astype(np.float32)

    # Calculate the NDVI formula.
    numerator = subtract(np_nir_as32, np_colour_as32)
    denominator = add(np_nir_as32, np_colour_as32)
    result = divide(numerator, denominator)

    # Remove any out-of-bounds areas
    result[result == -0] = -99

    # Initialize a geotiff driver.
    geotiff = GetDriverByName('GTiff')

    # If the desired output is an int16, map the domain [-1,1] to [0,255], create an int16 geotiff with one band and
    # write the contents of the int16 NDVI calculation to it.  Otherwise, create a float32 geotiff with one band and
    # write the contents of the float32 NDVI calculation to it.
    if data_type == gdal.GDT_UInt16:
        ndvi_int8 = multiply((result + 1), (2**7 - 1))
        output = geotiff.Create(out_tiff, in_cols, in_rows, 1, gdal.GDT_Byte)
        output_band = output.GetRasterBand(1)
        output_band.SetNoDataValue(-99)
        output_band.WriteArray(ndvi_int8)
    elif data_type == gdal.GDT_Float32:
        output = geotiff.Create(out_tiff, in_cols, in_rows, 1, gdal.GDT_Float32)
        output_band = output.GetRasterBand(1)
        output_band.SetNoDataValue(-99)
        output_band.WriteArray(result)
    else:
        raise ValueError('Invalid output data type.  Valid types are gdal.UInt16 or gdal.Float32.')

    # Set the geographic transformation as the input.
    output.SetGeoTransform(in_geotransform)

    return None

In [17]:
# Seperate NIR and RED bands images into different folders and add path of parent directory
path = "/kaggle/input/msi-dji/MSI_DJI/" 
nir = sorted(glob(os.path.join(path, "NIR/*")))
red = sorted(glob(os.path.join(path, "RED/*")))
print(f"Total NIR images:  {len(nir)} - Total RED images: {len(red)}")

Total NIR images:  86 - Total RED images: 86


In [27]:
# set i to the number of images in dataset
for i in range(0, 86):
    # Open NIR image and get its only band.
    nir_tiff = Open(nir[i])
    nir_band = nir_tiff.GetRasterBand(1)

    # Open red image and get its only band.
    red_tiff = Open(red[i])
    red_band = red_tiff.GetRasterBand(1)

    # Get the rows and cols from one of the images (both should always be the same)
    rows, cols, geotransform = nir_tiff.RasterYSize, nir_tiff.RasterXSize, nir_tiff.GetGeoTransform()
    #     print(geotransform)

    # Set an output for a 16-bit unsigned integer (0-255)
    out_tiff_int16 = f"{i+1}"+r"_NDVI_INT16.tif"

    # Set the output for a 32-bit floating point (-1 to 1)
    out_tiff_float32 = f'{i+1}'+r'_NDVI_FLOAT32.tif'

    # Run the function for unsigned 16-bit integer
    ndvi(nir_band, red_band, rows, cols, geotransform, out_tiff_int16, gdal.GDT_UInt16)

    # Run the function for 32-bit floating point
    ndvi(nir_band, red_band, rows, cols, geotransform, out_tiff_float32, gdal.GDT_Float32)

    print('Generating NDVI ',i+1)

done  0
done  1
done  2
done  3
done  4
done  5
done  6
done  7
done  8
done  9
done  10
done  11
done  12
done  13
done  14
done  15
done  16
done  17
done  18
done  19
done  20
done  21
done  22
done  23
done  24
done  25
done  26
done  27
done  28
done  29
done  30
done  31
done  32
done  33
done  34
done  35
done  36
done  37
done  38
done  39
done  40
done  41
done  42
done  43
done  44
done  45
done  46
done  47
done  48
done  49
done  50
done  51
done  52
done  53
done  54
done  55
done  56
done  57
done  58
done  59
done  60
done  61
done  62
done  63
done  64
done  65
done  66
done  67
done  68
done  69
done  70
done  71
done  72
done  73
done  74
done  75
done  76
done  77
done  78
done  79
done  80
done  81
done  82
done  83
done  84
done  85


In [24]:
# # delete file
# os.remove("/kaggle/working/NDVI_INT16.tif")

In [28]:
!zip -r /kaggle/working/ndvi.zip /kaggle/working/

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/76_NDVI_INT16.tif (deflated 22%)
  adding: kaggle/working/11_NDVI_INT16.tif (deflated 20%)
  adding: kaggle/working/5_NDVI_INT16.tif (deflated 20%)
  adding: kaggle/working/54_NDVI_INT16.tif (deflated 21%)
  adding: kaggle/working/21_NDVI_FLOAT32.tif (deflated 14%)
  adding: kaggle/working/74_NDVI_INT16.tif (deflated 21%)
  adding: kaggle/working/59_NDVI_FLOAT32.tif (deflated 13%)
  adding: kaggle/working/29_NDVI_FLOAT32.tif (deflated 15%)
  adding: kaggle/working/30_NDVI_FLOAT32.tif (deflated 13%)
  adding: kaggle/working/40_NDVI_FLOAT32.tif (deflated 14%)
  adding: kaggle/working/1_NDVI_FLOAT32.tif (deflated 16%)
  adding: kaggle/working/69_NDVI_FLOAT32.tif (deflated 14%)
  adding: kaggle/working/49_NDVI_INT16.tif (deflated 21%)
  adding: kaggle/working/25_NDVI_FLOAT32.tif (deflated 16%)
  adding: kaggle/working/63_NDVI_FLOAT32.tif (deflated 13%)
  adding: kaggle/working/45_NDVI_INT16.tif (deflated 22%)
  adding: kaggle/w